In [1]:
%load_ext autoreload
%autoreload 2

%load_ext dotenv
%dotenv

# STAC metadata generation

In this notebook we are going to generate the STAC metadata of our dataset, converting it from a Q0 dataset to a [Q1 dataset](../00_eotdl.ipynb). 

The [STAC](https://stacspec.org/en) specification is a common language to describe geospatial information, so it can more easily be worked with, indexed, and discovered. Is a standarized way to expose, host, ingest and access geospatial collections that has been adopted as the EOTDL standard metadata format. For further information, check their [website](https://stacspec.org/en).

In order to facilitate the STAC generation, which can be painful and time-expensive, the EOTDL environment has several open source tools that make the process much more straightforward. Let's dive into them!

The first thing we have to understand is that the process starts with a `STACDataFrame`. This `STACDataFrame` is an interface between the images and the STAC catalogs and collections, with some variables that we can define and customise to ensure that the STAC metadata has the information we want, like `extensions`, which defines the [STAC extensions](https://stac-extensions.github.io/) that the image must have, or `bands`, with the bands we want to get from the image, if any.

Let's see the example below.

In [2]:
import pandas as pd

sample_df = pd.read_csv('data/sample_stacdataframe.csv')
sample_df

,image,label,ix,collection,extensions,bands
0,data/sentinel_2/Boadella_2019-06-07/sentinel-2...,sentinel-2-l2a,0,data/sentinel_2/source,"('proj', 'raster')","('B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B0..."
1,data/sentinel_2/Boadella_2019-06-02/sentinel-2...,sentinel-2-l2a,0,data/sentinel_2/source,"('proj', 'raster')","('B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B0..."
2,data/sentinel_2/Boadella_2019-07-02/sentinel-2...,sentinel-2-l2a,0,data/sentinel_2/source,"('proj', 'raster')","('B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B0..."
3,data/sentinel_2/Boadella_2019-06-17/sentinel-2...,sentinel-2-l2a,0,data/sentinel_2/source,"('proj', 'raster')","('B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B0..."
4,data/sentinel_2/Boadella_2019-06-27/sentinel-2...,sentinel-2-l2a,0,data/sentinel_2/source,"('proj', 'raster')","('B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B0..."


This is a sample `STACDataFrame` already generated for our workshop. Here we can see a lot of information:
- image: the path to every image.
- label: the label assigned to every image. 
- ix: the index of the label.
- collection: the collection which the image belongs to. 
- extensions: a list with the STAC extensions we want the image STAC item to have.
- bands: a list of band we want the image STAC item to have.

Now we have seen this, let's generate the STAC metadata for our dataset. Don't worry, we are going to explain it step by step!

First of all, we need to import the `STACGenerator` class.

In [3]:
from eotdl.curation.stac.stac import STACGenerator

The `STACGenerator` class is the entry point and the STAC generation class, where magic happens. Before we declare it, we need to understand the parameters we can give to it:
- `image_format`: the extension of the images. Could be `png`, `jpg` and so on. By default is `tiff`.
- `catalog_type`: the STAC Catalog type. It is a specification defined [here](https://pystac.readthedocs.io/en/0.4/concepts.html#catalog-types). By default is `SELF_CONTAINED`. 
- `item_parser`: the item_parser defines the strategy that must be followed to search for satellite images within the folder. We have defined 2 item_parser strategies, and new ones can be added as needed. The strategies that are implemented right now are the following.
    - `StructuredParser`: this strategy is used when the images are each contained within a folder, so that the name of the item will be the name of the folder. We will use this strategy when using the EOTDL to download the dataset images, as it will always format the folder structure the same way. As this is what we have done, it is the strategy that we will use for the use case of this workshop, as every folder contains a single image.
    
    <p align="center">
        <img src="images/structured_parser.png" alt="Structured parser typical folder structure" style="height:170px; width:200px;"/>
    </p>
    
    - `UnestructuredParser`: this strategy is used when there are multiple images contained in the same folder.

    <p align="center">
        <img src="images/unestructured_parser.png" alt="Structured parser typical folder structure" style="height:200px; width:200px;"/>
    </p>
    
- `assets_generator`: the assets_generator parameter defines the strategy to follow with the generation of assets from each image. In this way, it could be the case that from a Sentinel-2 image we want to extract all its bands as assets, or simply extract the RGB bands, or not extract any as assets. By default, three strategies have been established, which can be expanded according to needs.

    - `STACAssetGenerator`: does not extract new assets from the image bands, so a single asset is generated for the image.
    - `BandsAssetGenerator`: from the original image it creates a new file for each band established in the 'bands` column, deleting the original file. An asset is added to the STAC item for each band.
    - `ExtractedAssets`: indicates that the bands of an image have already been extracted as independent files, so it creates an item for each image taking the files as assets.
    

- `labeling_strategy`: the `labeling_strategy` parameter defines the strategy to extract a label from the filename of an image, to assign a label to it. By default, we have implemented 2 strategies:

    - `UnlabeledStrategy`: we will use it when the images do not have a label that identifies them or that has been placed on purpose. It is the one we will use in our case, since the filename is simply the name of the constellation, and it is the default option.
    - `LabeledStrategy`: we will use it when the images are labeled with labels in their filenames. An example would be that in a folder the images were called, for example, River_1.png, River_2.png, River_3.png, and so on. The file name must be the pattern <label>_<number>.


For the specific case of our workshop, we will take into account the following:
- The images have been downloaded using the EOTDL and are each in a folder, so as `item_parser` we will use `StructuredParser`.
- In this case we will extract all the bands from the image as assets, so as `asset_generator` we will use `BandsAssetGenerator`, which is the default option.
- The images do not have labels, so we will use the default option, `UnlabeledStrategy`.

In [3]:
from eotdl.curation.stac.stac import STACGenerator
from eotdl.curation.stac.assets import BandsAssetGenerator
from eotdl.curation.stac.parsers import StructuredParser
from eotdl.curation.stac.labeling import UnlabeledStrategy

stac_generator = STACGenerator(item_parser=StructuredParser, 
                               assets_generator=BandsAssetGenerator, 
                               labeling_strategy=UnlabeledStrategy
                               )

If we now decide to generate a `STACDataFrame` from the folder with the images it will do fine, but it will be incomplete.

In [4]:
df = stac_generator.get_stac_dataframe('data/sentinel_2')
df.head()

,image,label,ix,collection,extensions,bands
0,data/sentinel_2/Boadella_2019-06-07/sentinel-2...,sentinel-2-l2a,0,data/sentinel_2/source,None,None
1,data/sentinel_2/Boadella_2019-06-02/sentinel-2...,sentinel-2-l2a,0,data/sentinel_2/source,None,None
2,data/sentinel_2/Boadella_2019-07-02/sentinel-2...,sentinel-2-l2a,0,data/sentinel_2/source,None,None
3,data/sentinel_2/Boadella_2019-06-17/sentinel-2...,sentinel-2-l2a,0,data/sentinel_2/source,None,None
4,data/sentinel_2/Boadella_2019-06-27/sentinel-2...,sentinel-2-l2a,0,data/sentinel_2/source,None,None


A key feature is the `label` column. Using the label of every image we are going to assign parameters like the STAC extensions that this image's item is going to have, or the bands we want to extract using the `BandsAssetGenerator`. We can obtain the existing labels in the STACDataFrame before adding new information.

In [5]:
labels = df.label.unique().tolist()
labels

['sentinel-2-l2a']

Starting from the `sentinel-2-l2a` label we are going to define several parameters:

- As STAC extensions we are going to implement the [proj](https://github.com/stac-extensions/projection), [raster](https://github.com/stac-extensions/raster) and [eo](https://github.com/stac-extensions/eo) STAC extensions. Note: the supported extensiones are `('eo', 'sar', 'proj', 'raster')`.
- As bands to extract from the Sentinel-2 image, we are going to extract all of them except B10.

To define these parameters for each label, we simply have to declare a dictionary.

In [8]:
extensions = {'sentinel-2-l2a': ('proj', 'raster', 'eo')}
bands = {'sentinel-2-l2a': ('B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B09', 'B11', 'B12')}

Now we are ready to generate a `STACDataFrame` with relevant information. Some extra parameters to take into account:
- `path`: is the root path where the images are located at. In our case is `data/sentinel_2`.
- `collections`: we can use this parameter to define the STAC collection to which we want each item with a specific label to go. There are several options:
    - The default option puts all the STAC items in a single collection called `source`.
    - The `*` option will consider folders located directly under the root folder as collections, so it will create a collection for each of them.

    <p align="center">
        <img src="images/collection.png" alt="* collection" style="height:170px; width:200px;"/>
    </p>

    - You can decide the collection you want an image to go to through its label, as we have seen in the case of extensions and bands. To give an example, we are going to define it like this.

In [6]:
collection = {'sentinel-2-l2a': 'boadella-sentinel-2'}

Let's generate the complete STACDataFrame!

In [9]:
df = stac_generator.get_stac_dataframe('data/sentinel_2', collections=collection, bands=bands, extensions=extensions)
df.head()

,image,label,ix,collection,extensions,bands
0,data/sentinel_2/Boadella_2019-06-07/sentinel-2...,sentinel-2-l2a,0,data/sentinel_2/boadella-sentinel-2,"(proj, raster, eo)","(B01, B02, B03, B04, B05, B06, B07, B08, B09, ..."
1,data/sentinel_2/Boadella_2019-06-02/sentinel-2...,sentinel-2-l2a,0,data/sentinel_2/boadella-sentinel-2,"(proj, raster, eo)","(B01, B02, B03, B04, B05, B06, B07, B08, B09, ..."
2,data/sentinel_2/Boadella_2019-07-02/sentinel-2...,sentinel-2-l2a,0,data/sentinel_2/boadella-sentinel-2,"(proj, raster, eo)","(B01, B02, B03, B04, B05, B06, B07, B08, B09, ..."
3,data/sentinel_2/Boadella_2019-06-17/sentinel-2...,sentinel-2-l2a,0,data/sentinel_2/boadella-sentinel-2,"(proj, raster, eo)","(B01, B02, B03, B04, B05, B06, B07, B08, B09, ..."
4,data/sentinel_2/Boadella_2019-06-27/sentinel-2...,sentinel-2-l2a,0,data/sentinel_2/boadella-sentinel-2,"(proj, raster, eo)","(B01, B02, B03, B04, B05, B06, B07, B08, B09, ..."


It looks good! We have all we need to generate the STAC metadata. We only have to give the catalog an `id`, a `description` and an `output_folder`!

In [10]:
stac_generator.generate_stac_metadata(id='boadella-dataset',
                                      description='Boadella dataset',
                                      output_folder='data/sentinel_2_stac')

Generating boadella-sentinel-2 collection...


100%|██████████| 5/5 [00:00<00:00, 47.99it/s]

Validating and saving catalog...
Success!


Voilà! We have generated the STAC metadata from our Q0 dataset, converting it into a Q1 dataset!

To sum up this section, we have learned the different parameters we need to know and understand to generate STAC metadata from a dataset in a easy way.

Let's continue in the `zz_notebook` notebook!